In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
# use get_db_url function to connect to the codeup db
from env import get_db_url

# custom module imports
import acquire as aq

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [3]:
df = aq.get_zillow_data()

In [15]:
df.head()

,parcelid,heatingorsystemtypeid,architecturalstyletypeid,id,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,propertylandusetypeid,propertylandusedesc,architecturalstyledesc,heatingorsystemdesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,logerror,transactiondate
0,14172985,6.0,7.0,2874473,1.0,None,2.5,4.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.007491,2017-01-03
1,14081891,6.0,7.0,1935368,NaN,None,2.5,3.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.011913,2017-01-03
2,14633780,6.0,8.0,803551,11.0,None,1.0,2.0,None,None,...,266,Condominium,Conventional,Forced air,None,None,6.0,Frame,0.067501,2017-01-04
3,14531476,18.0,7.0,1325329,11.0,None,2.0,2.0,None,None,...,261,Single Family Residential,Contemporary,Radiant,None,None,6.0,Frame,0.016042,2017-01-04
4,14663359,6.0,7.0,2101221,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,0.027668,2017-01-05


### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
aq.summarize(df)

----------------------
Dataframe head
   parcelid  heatingorsystemtypeid  architecturalstyletypeid       id  \
0  14172985                    6.0                       7.0  2874473   
1  14081891                    6.0                       7.0  1935368   
2  14633780                    6.0                       8.0   803551   

   airconditioningtypeid basementsqft  bathroomcnt  bedroomcnt  \
0                    1.0         None          2.5         4.0   
1                    NaN         None          2.5         3.0   
2                   11.0         None          1.0         2.0   

  buildingclasstypeid buildingqualitytypeid  ...  propertylandusetypeid  \
0                None                  None  ...                    261   
1                None                  None  ...                    261   
2                None                  None  ...                    266   

         propertylandusedesc architecturalstyledesc  heatingorsystemdesc  \
0  Single Family Residentia

AttributeError: 'DataFrame' object has no attribute 'dtype'

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [75]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 
                                 'percent_rows_missing': prcnt_miss})
    return cols_missing

In [91]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
parcelid,0,0.000000
heatingorsystemtypeid,1,0.485437
architecturalstyletypeid,0,0.000000
id,0,0.000000
airconditioningtypeid,88,42.718447
...,...,...
storydesc,206,100.000000
typeconstructiontypeid,7,3.398058
typeconstructiondesc,7,3.398058
logerror,0,0.000000


### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [30]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis = 1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 
                                 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index().set_index('num_cols_missing')
    return rows_missing

In [90]:
nulls_by_row(df)

,percent_cols_missing,num_rows
num_cols_missing,,
23,33.33333333333333,2
24,34.78260869565217,11
25,36.231884057971016,32
26,37.68115942028986,61
27,39.130434782608695,57
28,40.57971014492754,23
29,42.028985507246375,9
30,43.47826086956522,5
31,44.927536231884055,3


## Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [50]:
df.head()

,parcelid,heatingorsystemtypeid,architecturalstyletypeid,id,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,propertylandusetypeid,propertylandusedesc,architecturalstyledesc,heatingorsystemdesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,logerror,transactiondate
0,14172985,6.0,7.0,2874473,1.0,None,2.5,4.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.007491,2017-01-03
1,14081891,6.0,7.0,1935368,NaN,None,2.5,3.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.011913,2017-01-03
2,14633780,6.0,8.0,803551,11.0,None,1.0,2.0,None,None,...,266,Condominium,Conventional,Forced air,None,None,6.0,Frame,0.067501,2017-01-04
3,14531476,18.0,7.0,1325329,11.0,None,2.0,2.0,None,None,...,261,Single Family Residential,Contemporary,Radiant,None,None,6.0,Frame,0.016042,2017-01-04
4,14663359,6.0,7.0,2101221,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,0.027668,2017-01-05


In [51]:
df[(df.bedroomcnt > 2) & (df.bathroomcnt> 1.5)]

,parcelid,heatingorsystemtypeid,architecturalstyletypeid,id,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,propertylandusetypeid,propertylandusedesc,architecturalstyledesc,heatingorsystemdesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,logerror,transactiondate
0,14172985,6.0,7.0,2874473,1.0,None,2.5,4.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.007491,2017-01-03
1,14081891,6.0,7.0,1935368,NaN,None,2.5,3.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,0.011913,2017-01-03
4,14663359,6.0,7.0,2101221,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,0.027668,2017-01-05
6,14687147,NaN,8.0,1833183,NaN,None,2.5,3.0,None,None,...,266,Condominium,Conventional,None,None,None,6.0,Frame,0.067520,2017-01-05
8,14656443,1.0,7.0,2563124,NaN,None,2.0,3.0,None,None,...,266,Condominium,Contemporary,Baseboard,None,None,6.0,Frame,0.153751,2017-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,14656461,6.0,7.0,833461,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,0.081789,2017-08-30
193,14487731,6.0,7.0,2696785,13.0,None,2.0,3.0,None,None,...,261,Single Family Residential,Contemporary,Forced air,None,None,6.0,Frame,-0.045950,2017-08-31
198,14669224,6.0,7.0,863387,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,-0.004824,2017-09-01
201,14650994,6.0,7.0,2518405,NaN,None,2.5,3.0,None,None,...,266,Condominium,Contemporary,Forced air,None,None,6.0,Frame,0.070490,2017-09-07


In [58]:
df.propertylandusedesc[df.bedroomcnt == df.bedroomcnt.max()]

97    Single Family Residential
Name: propertylandusedesc, dtype: object

In [59]:
df.propertylandusedesc[df.bedroomcnt == df.bedroomcnt.min()]

16                   Condominium
26                   Condominium
28     Single Family Residential
38     Single Family Residential
49                   Condominium
50                   Condominium
54                   Condominium
59                   Condominium
63                   Condominium
77                   Condominium
88                   Condominium
100                  Condominium
130                  Condominium
140                  Condominium
143    Single Family Residential
153                  Condominium
169    Single Family Residential
195                  Condominium
196                  Condominium
204    Single Family Residential
Name: propertylandusedesc, dtype: object

In [60]:
df.bedroomcnt.min()

1.0

### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [92]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df
                 
                 
def handle_missing_values(df, prop_required_column=0.5 , prop_required_row=0.75):
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    return df

# combined in one function
def data_prep(df, cols_to_remove=[], prop_required_column=0.5, prop_required_row=0.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df
